In [1]:
import copy
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm

from stanza.server import CoreNLPClient

from projectmir.xmldocument_1 import XMLDocument
from projectmir.functions import find_identifier_definition

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]

doc = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    documents_path = p_path + documents_ID + '.html'
    doc_ = find_identifier_definition(documents_path)
    doc[i] = doc_

save_filename = 'doc' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
with open(save_filename, 'wb') as f:
    pickle.dump(doc, f)


2it [01:28, 44.42s/it]


elapsed time : 0.0000381 seconds ---
preprocessed document
elapsed time : 0.0055511 seconds ---
Number of math components is 50
elapsed time : 0.0782993 seconds ---
elapsed time : 0.1246581 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-b6bff37db5954cca.props -preload tokenize,ssplit,pos
elapsed time : 2.0794942 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-592084389ed54875.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
extract_noun_phrases
elapsed time : 45.2497590 seconds ---
elapsed time : 

In [3]:
load_filename = 'doc_23Aug2020_152844.pkl'
# load_filename = 'doc.pkl'
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)
# print(doc[0].path)
# print(doc[0].identifiers)

In [4]:
doc0 = doc[0]

In [34]:
# sentence segmentation using stanza
import stanza
# stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize')
doc_sentence_segmentated = nlp(doc0.text)
for i, sentence in enumerate(doc_sentence_segmentated.sentences):
    # print(f'========== Sentence {i+1} tokens =======')
    # print(*[f'id: {token.id}\ttext:: {token.text}' for token in sentence.tokens], sep='\n')
    print(f'{i}:', *[f'{token.text}' for token in sentence.tokens])

2020-08-23 16:56:50 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-23 16:56:50 INFO: Use device: cpu
2020-08-23 16:56:50 INFO: Loading: tokenize
2020-08-23 16:56:50 INFO: Done loading processors!


0: Process dynamics and control
1: Shota Kato
2: June 8 , 2020
3: 1 DYNAMIC MODELS OF REPRESENTATIVE PROCESSES
4: 1.1 Stirred - Tank Heating Process : Constant Holdup
5: Consider the stirred - tank heating system shown in Fig. 2.3 .
6: The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001 .
7: The tank contents are agitated and heated using an electrical heater that provides a heating rate , MATH0002 .
8: A dynamic model will be developed based on the following assumptions :
9: 1 .
10: Perfect mixing : thus , the exit temperature MATH0003 is also the temperature of the tank contents .
11: 2 .
12: The inlet and outlet flow rates are equal ; thus , MATH0000 = MATH0004 and the liquid holdup MATH0005 is constant .
13: 3 .
14: The density MATH0006 and heat capacity MATH0007 of the liquid are assumed to be constant .
15: Thus , their temperature dependence is neglected .
16: 4 .
17: Heat losses are negligible .
18: In general 

In [41]:
sentence_text = ' '.join([f'{token.text}' for token in sentence.tokens])

In [42]:
print(sentence_text)

Generated on Mon Jun 8 18:33:41 2020 by LaTeXML


In [6]:
%%time
# sentetnce segmentation using CoreNLP
with CoreNLPClient(
    annotators=['tokenize', 'ssplit'],#, 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'],
    timeout=30000,
    memory='16G',
    endpoint='http://localhost:9010') as client:
    ann = client.annotate(doc0.text)
# # get the first sentence
# sentence = ann.sentence[0]
# # print(*[f'id: {token.tokenBeginIndex}\ttext:: {token.word}' for token in sentence.token], sep='\n')
# print(*[f'{token.word}' for token in sentence.token])
for i, sentence in enumerate(ann.sentence):
    # print(f'========== Sentence {i+1} tokens =======')
    # print(*[f'id: {token.tokenBeginIndex}\ttext:: {token.word}' for token in sentence.token], sep='\n')
    print(f'{i}:\t', *[f'{token.word}' for token in sentence.token])


Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9010 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-46a0e06b1fdf4238.props -preload tokenize,ssplit
0:	 Process dynamics and control Shota Kato June 8 , 2020 1 DYNAMIC MODELS OF REPRESENTATIVE PROCESSES 1.1 Stirred - Tank Heating Process : Constant Holdup Consider the stirred - tank heating system shown in Fig. 2.3 .
1:	 The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001 .
2:	 The tank contents are agitated and heated using an electrical heater that provides a heating rate , MATH0002 .
3:	 A dynamic model will be developed based on the following assumptions : 1 .
4:	 Perfect mixing : thus , the exit temperature MATH0003 is also the temperature of the tank contents .
5:	 2 .
6:	 The inlet and outlet 

In [43]:
# https://github.com/stanfordnlp/stanza/issues/288
# get noun phrases with tregex
def noun_phrases(_client, _text, _annotators=None):
    pattern = 'NP'
    matches = _client.tregex(_text, pattern, annotators=_annotators)
    return [sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]
    # print('\n'.join(['\t' + sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]))
with CoreNLPClient(timeout=30000, memory='16G', endpoint='http://localhost:9012') as client:
    sentence_list = doc0.sentences[0]
    sentence = sentence_list[0]
    print(sentence)
    noun_phrase_list = noun_phrases(client, sentence, _annotators='tokenize,ssplit,pos,lemma,parse')



Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9012 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-895089c8d7174406.props -preload tokenize,ssplit,pos,lemma,ner,depparse
The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.


In [44]:
[i.lstrip() for i in noun_phrase_list]

['The liquid inlet stream',
 'a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001',
 'a single component',
 'a mass flow rate MATH0000 and an inlet temperature MATH0001',
 'a mass flow rate MATH0000',
 'an inlet temperature MATH0001']

In [9]:
doc0.sentences[0][0]

'The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.'

In [7]:
for identifier, description in zip(doc0.identifiers, doc0.candidate_noun_phrases_list):
    print(identifier)
    for description_ in description:
        for description__ in description_:
            def remove_stopwords(description):
                description_output = description
                # stopwords = []
                # stopwords are tags used in tregex and common stopwords.
                stopwords = ['\n', '(', ')', 'NP', 'DT', 'JJ', 'NN', 'PP', 'IN', 'CC']
                # 参考：https://qiita.com/syunyo/items/2c1ce1d765f46a5c1d72
                
                for stopword in stopwords:
                    description_output = description_output.replace(stopword, '')
                    description_output = description_output.replace('  ', ' ')
                return description_output
            print(remove_stopwords(description__))
            # print(description__.replace('\n', ''))

    print('\n')

w_i
(NP (DT The) (JJ liquid) (NN inlet) (NN stream))
(NP
  (NP (DT a) (JJ single) (NN component))
  (PP (IN with)
    (NP
      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
      (CC and)
      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))
(NP (DT a) (JJ single) (NN component))
(NP
  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
  (CC and)
  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))
(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))
(NP (DT The)
  (NP
    (NP (NN inlet))
    (CC and)
    (NP (NN outlet) (NN flow) (NNS rates))))
(NP
  (NP (NN inlet))
  (CC and)
  (NP (NN outlet) (NN flow) (NNS rates)))
(NP (NN inlet))
(NP (NN outlet) (NN flow) (NNS rates))
(NP
  (NP (NN MATH0000) (JJ =) (NN MATH0004))
  (CC and)
  (NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005)))
(NP (NN MATH0000) (JJ =) (NN MATH0004))
(NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005))


T_i
(NP (DT The) (JJ li

In [5]:
with open('./doc0_text.txt', 'w') as f:
    f.write(doc0.text)
with open('./doc0_body.txt', 'w') as f:
    f.write(doc0.body)

In [60]:
(doc0.candidate_noun_phrases_list[0][0] + doc0.candidate_noun_phrases_list[0][1])

13

In [20]:
with open('./doc0_113130.txt', 'w') as f:
    f.write('\n'.join(doc0.sentences[0]))

In [ ]:
from stanza.server import CoreNLPClient


In [104]:
def extract_noun_phrases(identifiers, sentences):
    candidate_noun_phrases_list = [[]] * len(identifiers)
    pattern = 'NP'
    with CoreNLPClient(
            annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'],
            timeout=600000, memory='16G') as client:
        for i, sentence in enumerate(sentences):
            if sentence:
                candidate_noun_phrases_list_ = []
                for text in sentence:
                    matches = client.tregex(text, pattern)
                    matches = matches['sentences'][0]
                    candidates = [matches[match]['match'].rstrip()
                                    for match in matches]
                    candidate_noun_phrases_list_ += candidates

                print('##################')
                print(type(candidate_noun_phrases_list_))
                print(candidate_noun_phrases_list_)
                print('##################')

                candidate_noun_phrases_list[i] = candidate_noun_phrases_list_
    return candidate_noun_phrases_list

In [107]:
xx = extract_noun_phrases(doc0.identifiers[:1], doc0.sentences[:1])

Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c7c45c2a039b4d1b.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
##################
<class 'list'>
['(NP (DT The) (JJ liquid) (NN inlet) (NN stream))', '(NP\n  (NP (DT a) (JJ single) (NN component))\n  (PP (IN with)\n    (NP\n      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n      (CC and)\n      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))', '(NP (DT a) (JJ single) (NN component))', '(NP\n  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n  (CC and)\n  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))', '(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))', '(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))', '(NP (DT The)\n  (NP\n    (NP

In [112]:
print('\n\n'.join(xx[0]))

(NP (DT The) (JJ liquid) (NN inlet) (NN stream))

(NP
  (NP (DT a) (JJ single) (NN component))
  (PP (IN with)
    (NP
      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
      (CC and)
      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))

(NP (DT a) (JJ single) (NN component))

(NP
  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
  (CC and)
  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))

(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))

(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))

(NP (DT The)
  (NP
    (NP (NN inlet))
    (CC and)
    (NP (NN outlet) (NN flow) (NNS rates))))

(NP
  (NP (NN inlet))
  (CC and)
  (NP (NN outlet) (NN flow) (NNS rates)))

(NP (NN inlet))

(NP (NN outlet) (NN flow) (NNS rates))

(NP
  (NP (NN MATH0000) (JJ =) (NN MATH0004))
  (CC and)
  (NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005)))

(NP (NN MATH0000) (JJ =) (NN MATH0004))

(NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005))


In [135]:
doc0.tagged_sentence_list[0]

[[('The', 'DT'),
  ('liquid', 'JJ'),
  ('inlet', 'NN'),
  ('stream', 'NN'),
  ('consists', 'VBZ'),
  ('of', 'IN'),
  ('a', 'DT'),
  ('single', 'JJ'),
  ('component', 'NN'),
  ('with', 'IN'),
  ('a', 'DT'),
  ('mass', 'NN'),
  ('flow', 'NN'),
  ('rate', 'NN'),
  ('MATH0000', 'NN'),
  ('and', 'CC'),
  ('an', 'DT'),
  ('inlet', 'JJ'),
  ('temperature', 'NN'),
  ('MATH0001', 'NN'),
  ('.', '.')],
 [('The', 'DT'),
  ('inlet', 'NN'),
  ('and', 'CC'),
  ('outlet', 'NN'),
  ('flow', 'NN'),
  ('rates', 'NNS'),
  ('are', 'VBP'),
  ('equal', 'JJ'),
  (';', ':'),
  ('thus', 'RB'),
  (',', ','),
  ('MATH0000', 'NN'),
  ('=', 'JJ'),
  ('MATH0004', 'NN'),
  ('and', 'CC'),
  ('the', 'DT'),
  ('liquid', 'JJ'),
  ('holdup', 'NN'),
  ('MATH0005', 'NN'),
  ('is', 'VBZ'),
  ('constant', 'JJ'),
  ('.', '.')]]

In [121]:
doc0.candidate_noun_phrases_list[0][0]
# -> 最初のidentifierに対するnoun candidate
# candidate にidentifierとの距離を埋め込む．-> candidate.extract_noun_phraseとtagged_sentence_listを比較して，長さを計算する．


['(NP (DT The) (JJ liquid) (NN inlet) (NN stream))',
 '(NP\n  (NP (DT a) (JJ single) (NN component))\n  (PP (IN with)\n    (NP\n      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n      (CC and)\n      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))',
 '(NP (DT a) (JJ single) (NN component))',
 '(NP\n  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n  (CC and)\n  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))',
 '(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))',
 '(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))']

In [134]:
# 最初のidentifier[0]が含まれる1つ目の文章[0]の1番目のcandidate[0]
doc0.candidate_noun_phrases_list[0][0][0].replace('NP', '')

'( (DT The) (JJ liquid) (NN inlet) (NN stream))'

In [123]:
doc0.sentences[0][0]

'The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.'

In [ ]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs